In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk, string
from nltk.stem.porter import PorterStemmer
import re

#nltk.download('stopwords')
#nltk.download('punkt')


In [2]:
documents = [open(f).read() for f in ['./lorem1.txt', './lorem2.txt', './lorem3.txt']]
tfidf = TfidfVectorizer().fit_transform(documents)
# no need to normalize, since Vectorizer will return normalized tf-idf
pairwise_similarity = tfidf * tfidf.T

In [6]:
def initial_clean(text):
    """
    Function to clean text of websites, email addresess and any punctuation
    We also lower case the text
    """
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    
    text = re.sub("((\S+)?(http(s)?)(\S+))|((\S+)?(www)(\S+))|((\S+)?(\@)(\S+)?)", " ", text)
    text = text.lower() # lower case the text
    text = text.translate(remove_punctuation_map)
    text = nltk.word_tokenize(text)
    return text

stop_words = stopwords.words('english')
def remove_stop_words(text):
    """
    Function that removes all stopwords from text
    """
    return [word for word in text if word not in stop_words]

stemmer = PorterStemmer()
def stem_words(text):
    """
    Function to stem words, so plural and singular are treated the same
    """
    try:
        text = [stemmer.stem(word) for word in text]
        text = [word for word in text if len(word) > 1] # make sure we have no 1 letter words
    except IndexError: # the word "oed" broke this, so needed try except
        pass
    return text

def apply_all(text):
    """
    This function applies all the functions above into one
    """
    return stem_words(remove_stop_words(initial_clean(text)))

In [7]:
tfidf = TfidfVectorizer(tokenizer=apply_all).fit_transform(documents)
# no need to normalize, since Vectorizer will return normalized tf-idf
pairwise_similarity = tfidf * tfidf.T
pairwise_similarity.toarray()

array([[1.        , 0.        , 0.99630252],
       [0.        , 1.        , 0.        ],
       [0.99630252, 0.        , 1.        ]])

In [4]:
import textract
text = textract.process("/Users/stefan/Nextcloud/Documents/ebooks/deeplearningbook.pdf")